## Imports

In [ ]:
import warnings
from tqdm import tqdm

import pandas as pd
import geopandas as gpd

from potentiel_solaire.database.queries import get_high_priority_schools, get_regions
from potentiel_solaire.features.roof_attributes import segmentation_toits, recuperation_mns

from potentiel_solaire.constants import RESULTS_FOLDER, DEFAULT_CRS

warnings.filterwarnings("ignore")

## Constants

In [ ]:
DEFAULT_SRS_METERS = 'EPSG:2154'  # used for WMS requests
DEFAULT_CRS_METERS = int(DEFAULT_SRS_METERS.split(":")[1])

# Overwrite the default projections with specific ones for DROMS
SRS_BY_REGION = {
    "01": "EPSG:5490",  # Guadeloupe
    "02": "EPSG:5490",  # Martinique
    # "03": "EPSG:2972", TODO: find correct srs for Guyane for WMS requests
    "04": "EPSG:2975",  # La Réunion
    "06": "EPSG:4471",  # Mayotte
}

NB_SAMPLES = 1000
MIN_SEGMENT_SURFACE = 2

## Bâtiments des écoles prioritaires pour Greenpeace

In [ ]:
priotirized_schools = get_high_priority_schools()

priotirized_schools_buildings = []
for code_departement in priotirized_schools["code_departement"].unique():
    # Read results for the current department
    path_results = RESULTS_FOLDER / f"D{code_departement}_pipeline_results.gpkg"
    schools_buildings_dep = gpd.read_file(
        path_results, layer="solar_potential_of_schools_buildings"
    ).to_crs(DEFAULT_CRS_METERS)[[
        "cleabs_bat", 
        "identifiant_de_l_etablissement", 
        "surface_totale_au_sol", 
        "surface_utile", 
        "potentiel_solaire",
        "geometry"
    ]]

    # Get priotirized schools for the current department
    priotirized_schools_dep = priotirized_schools[priotirized_schools["code_departement"] == code_departement]

    # Filter priotirized schools buildings for the current department
    priotirized_schools_buildings_dep = schools_buildings_dep.merge(
        priotirized_schools_dep[["identifiant_de_l_etablissement", "code_region", "code_departement"]],
        on="identifiant_de_l_etablissement",
        how="inner"
    )

    # Append to the list
    priotirized_schools_buildings.append(priotirized_schools_buildings_dep)


# Concatenate all departements geodataframes
priotirized_schools_buildings = gpd.GeoDataFrame(
    pd.concat(priotirized_schools_buildings, ignore_index=True), 
    crs=DEFAULT_CRS_METERS
)

# Save the results
output_path = RESULTS_FOLDER / "priotirized_schools_buildings.gpkg"
priotirized_schools_buildings.to_file(
    output_path, 
    layer="results_with_simplified_method",
    driver="GPKG"
)

nb_priotirized_schools = len(priotirized_schools_buildings["identifiant_de_l_etablissement"].unique())
nb_priotirized_schools_buildings = len(priotirized_schools_buildings)
print(f"Nombre d'établissements scolaires prioritaires : {nb_priotirized_schools}")
print(f"Nombre de bâtiments d'établissements scolaires prioritaires : {nb_priotirized_schools_buildings}")

In [ ]:
selected_regions = get_regions()
selected_regions.remove("03")  # Exclude Guyane as WMS requests will fail

priotirized_schools_buildings_sampled = priotirized_schools_buildings[
    priotirized_schools_buildings["code_region"].isin(selected_regions)
].sample(n=NB_SAMPLES).reset_index(drop=True).copy()

nb_buildings_sampled = len(priotirized_schools_buildings_sampled)
print(f"Nombre de bâtiments d'établissements scolaires prioritaires échantillonnés : {nb_buildings_sampled}")

## Calculs en utilisant la segmentation des toits via les MNS

In [ ]:
def compute_roof_attributes_with_mns(
    building: gpd.GeoDataFrame, 
    min_surface : float, 
    crs: int,
    srs: str, 
    cleabs_bat: str,
    identifiant_de_l_etablissement: str,
    code_departement: str,
    code_region: str,
    longitude: float,
    latitude: float,
):      
    building = building.to_crs(crs)

    # Compute roof attributes using MNS
    result = segmentation_toits(
        recuperation_mns(
            zone_of_interest=building, 
            srs=srs
        ), 
        min_surface=min_surface
    )

    if result.empty:
        return result

    # Add building context
    result["cleabs_bat"] = cleabs_bat
    result["identifiant_de_l_etablissement"] = identifiant_de_l_etablissement
    result["code_departement"] = code_departement
    result["code_region"] = code_region
    result["longitude"] = longitude
    result["latitude"] = latitude

    # Bins of 5 degrees for slope and azimuth
    result["slope_bin_min"] = ((result["slope"] // 5) * 5).astype(int)
    result["slope_bin_max"] = result["slope_bin_min"] + 5
    result["azimut_bin_min"] = ((result["azimut"] // 5) * 5).astype(int)
    result["azimut_bin_max"] = result["azimut_bin_min"] + 5

    return result

In [ ]:
roof_segments = []
for cleabs_bat in tqdm(priotirized_schools_buildings_sampled["cleabs_bat"].unique()):
    building = priotirized_schools_buildings_sampled[priotirized_schools_buildings_sampled["cleabs_bat"] == cleabs_bat].copy()

    assert len(building) == 1, f"Multiple buildings found for cleabs_bat {cleabs_bat}"

    cleabs_bat = building["cleabs_bat"].values[0]
    identifiant_de_l_etablissement = building["identifiant_de_l_etablissement"].values[0]
    code_departement = building["code_departement"].values[0]
    code_region = building["code_region"].values[0]
    building_centroid = building.to_crs(DEFAULT_CRS).geometry.centroid
    longitude = building_centroid.x.values[0]
    latitude = building_centroid.y.values[0]

    srs = SRS_BY_REGION.get(code_region, DEFAULT_SRS_METERS)
    crs = int(srs.split(":")[1])

    building_result = compute_roof_attributes_with_mns(
        building=building,
        min_surface=MIN_SEGMENT_SURFACE,
        crs=crs,
        srs=srs,
        cleabs_bat=cleabs_bat,
        identifiant_de_l_etablissement=identifiant_de_l_etablissement,
        code_departement=code_departement,
        code_region=code_region,
        longitude=longitude,
        latitude=latitude,
    )

    if not building_result.empty:
        roof_segments.append(building_result)

# Concatenate all results
roof_segments = pd.concat(roof_segments, ignore_index=True)

## Sauvegarde des resultats

In [ ]:
output_path = RESULTS_FOLDER / "roof_segments.csv"
roof_segments.to_csv(
    output_path,
    index=False
)